In [2]:
import unsloth
from unsloth import FastLanguageModel
from datasets import load_dataset, Dataset

from tqdm import tqdm
import torch
import time
import pandas as pd
import numpy as np
from dotenv import load_dotenv

from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
from pathlib import Path
import torch
# Load the .env file
load_dotenv()

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/mxs220189/miniconda3/envs/pyllmpatch/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/mxs220189/miniconda3/envs/pyllmpatch/lib/python3.13/site-packages/torch/cuda/__init__.py:283: UserWarning: 
    Found GPU0 NVIDIA GB10 which is of cuda capability 12.1.
    Minimum and Maximum cuda capability supported by this version of PyTorch is
    (8.0) - (12.0)
    
  warnings.warn(


🦥 Unsloth Zoo will now patch everything to make training faster!


True

In [3]:
print(torch.cuda.is_available())

True


In [4]:
model_name="unsloth/Qwen2.5-Coder-32B-Instruct"

In [5]:
max_seq_length = 32768 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# # 4bit pre quantized models we support for 4x faster downloading + no OOMs.
# fourbit_models = [
#     "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
#     "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
#     "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
#     "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
#     "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
#     "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
#     "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
#     "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
#     "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
#     "unsloth/Phi-3-medium-4k-instruct",
#     "unsloth/gemma-2-9b-bnb-4bit",
#     "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
# ] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    # Can select any from the below:
    # "unsloth/Qwen2.5-0.5B", "unsloth/Qwen2.5-1.5B", "unsloth/Qwen2.5-3B"
    # "unsloth/Qwen2.5-14B",  "unsloth/Qwen2.5-32B",  "unsloth/Qwen2.5-72B",
    # And also all Instruct versions and Math. Coding verisons!
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    device_map={"": "cuda:0"}
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2026.1.2: Fast Qwen2 patching. Transformers: 4.57.3.
   \\   /|    NVIDIA GB10. Num GPUs = 1. Max memory: 119.699 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu130. CUDA: 12.1. CUDA Toolkit: 13.0. Triton: 3.6.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards: 100%|██████████| 4/4 [01:50<00:00, 27.71s/it]


In [6]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2026.1.2 patched 64 layers with 64 QKV layers, 64 O layers and 64 MLP layers.


In [7]:
tokenizer.add_eos_token = True
# tokenizer.pad_token_id = 0
# tokenizer.padding_side = "left"

In [8]:
model.config.use_cache = False

In [9]:
messages = [
    {
        "role": "system",
        "content": "You are a senior Python developer. Always return clean, correct Python code only WITHOUT adding any fences such as ```python <code> ```, or ``` <code> ``` to your response. Do not include any explanations or additional texts. Only provide the code without fences which is compilable."
    },
    {
        "role": "user",
        "content": "Write a Python function to compute the factorial of a number."
    }
]

In [10]:
print(model.device)

cuda:0


In [11]:

inputs = tokenizer.apply_chat_template(
	messages,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
).to(model.device)
outputs = model.generate(**inputs, max_new_tokens=8012)
print(tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:], skip_special_tokens=True))

def factorial(n):
    if n < 0:
        raise ValueError("Factorial is not defined for negative numbers")
    elif n == 0 or n == 1:
        return 1
    else:
        result = 1
        for i in range(2, n + 1):
            result *= i
        return result


In [12]:
def read_csv_file(file_name: str) -> pd.DataFrame:
    csv_path = Path.cwd() / file_name
    print(csv_path)
    return pd.read_csv(csv_path)

In [13]:
dataset_read = read_csv_file("../dataset/syntax_error_user_patches_finetune_dataset.csv")

/home/mxs220189/pylingual_collaboration/pylingual_download/code/finetuning/../dataset/syntax_error_user_patches_finetune_dataset.csv


In [14]:
dataset_read = dataset_read.dropna()
dataset_read = dataset_read[dataset_read["old_code_full"].str.strip() != ""]
dataset_read = dataset_read[dataset_read["new_code_full"].str.strip() != ""]

In [15]:
SYSTEM_PROMPT = (
    "You are an expert Python programmer and code repair specialist. "
    "Your task is to correct Python SYNTAX errors only.\n"
    "Make ONLY the minimal changes strictly required for the code to parse and compile successfully.\n"
    "Do NOT fix logic errors, runtime errors, or improve code quality unless required to resolve a syntax error.\n"
    "Do NOT refactor, reformat, or modify any code that is already syntactically valid.\n"
    "Preserve all error-free lines exactly as they appear in the original code.\n"
    "The final output must be a syntactically valid Python program.\n"
    "Output ONLY the corrected Python code.\n"
    "Do NOT include explanations, reasoning, comments, markdown, or formatting wrappers of any kind.\n"
    "Do NOT add any text before or after the code."
)


In [16]:
USER_PROMPT_TEMPLATE = (
    "Analyze the Python code snippet below and fix all syntax errors.\n\n"
    "Initial error message:\n"
    "{error_message}\n\n"
    "Initial code snippet:\n"
    "{code_snippet}\n\n"
    "If the code contains no syntax errors or the errors cannot be fixed, return the code unchanged."
)


In [17]:
USER_PROMPT_RETRY = (
    "Keeping in mind the initial code snippet provided and the code snippet from the previous response, "
    "along with the compilation error resulting from the previous response, make a new attempt to fix "
    "the initial snippet.\n\n"
    "Ensure your response only modifies the erroring lines while keeping the rest of the program the same.\n"
    "You must make the absolute minimum edit required to fix the errors.\n"
    "Use the original code as reference and keep all error-free lines unchanged.\n\n"
    "Previous compilation error:\n"
    "{prev_error_message}\n\n"
    "Original code snippet:\n"
    "{original_code}\n\n"
    "Previous (incorrect) response:\n"
    "{previous_response}"
)


In [18]:
def build_chat_messages(
    code_snippet: str,
    error_message: str,
    system_prompt: str,
    user_prompt_template: str,
) -> list[dict]:
    user_prompt = user_prompt_template.format(
        error_message=error_message,
        code_snippet=code_snippet,
    )

    messages = [
        {
            "role": "system",
            "content": system_prompt.strip(),
        },
        {
            "role": "user",
            "content": user_prompt.strip(),
        },
    ]

    return messages


In [19]:
def dataframe_to_chat_samples(
    df: pd.DataFrame,
    system_prompt: str,
    user_prompt_template: str,
    error_col: str = "syntactic_error_description",
) -> list[dict]:

    samples = []

    for _, row in df.iterrows():
        messages = build_chat_messages(
            code_snippet=row["old_code_full"].strip("\n"),
            error_message=row[error_col],
            system_prompt=system_prompt,
            user_prompt_template=user_prompt_template,
        )

        messages.append({
            "role": "assistant",
            "content": row["new_code_full"].strip("\n"),
        })

        samples.append({"messages": messages})

    return samples


In [20]:
def formatting_func(example):
    texts = []

    messages_field = example["messages"]

    # Handle batched input (list of conversations)
    if isinstance(messages_field, list) and messages_field and isinstance(messages_field[0], list):
        conversations = messages_field
    # Handle single example
    else:
        conversations = [messages_field]

    for messages in conversations:
        text = tokenizer.apply_chat_template(
            messages,                    # system / user / assistant turns
            tokenize=False,              # return text, not tokens
            add_generation_prompt=False, # training mode
        )
        texts.append(text)

    return texts


In [21]:
VALIDATION_RATIO = 0.1
RANDOM_SEED = 42

In [22]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(
    dataset_read,
    test_size=VALIDATION_RATIO,
    random_state=RANDOM_SEED,
    shuffle=True,
)


In [23]:
train_samples = dataframe_to_chat_samples(
    train_df,
    system_prompt=SYSTEM_PROMPT,
    user_prompt_template=USER_PROMPT_TEMPLATE,
)

In [24]:

val_samples = dataframe_to_chat_samples(
    val_df,
    system_prompt=SYSTEM_PROMPT,
    user_prompt_template=USER_PROMPT_TEMPLATE,
)

In [25]:
train_dataset = Dataset.from_list(train_samples)
val_dataset = Dataset.from_list(val_samples)


In [26]:
print(formatting_func(train_dataset[0]))

['<|im_start|>system\nYou are an expert Python programmer and code repair specialist. Your task is to correct Python SYNTAX errors only.\nMake ONLY the minimal changes strictly required for the code to parse and compile successfully.\nDo NOT fix logic errors, runtime errors, or improve code quality unless required to resolve a syntax error.\nDo NOT refactor, reformat, or modify any code that is already syntactically valid.\nPreserve all error-free lines exactly as they appear in the original code.\nThe final output must be a syntactically valid Python program.\nOutput ONLY the corrected Python code.\nDo NOT include explanations, reasoning, comments, markdown, or formatting wrappers of any kind.\nDo NOT add any text before or after the code.<|im_end|>\n<|im_start|>user\nAnalyze the Python code snippet below and fix all syntax errors.\n\nInitial error message:\nFile "/decompiler_workspace/5a50f193622344aa51b534ad73a7c6f1b897d82baa47d43486487360d024167c/decompiler_output/indented_0.py", l

In [27]:
from pprint import pprint

print("=== TRAIN SAMPLE ===")
pprint(train_dataset[0]["messages"])

print("=== VAL SAMPLE ===")
pprint(val_dataset[0]["messages"])


=== TRAIN SAMPLE ===
[{'content': 'You are an expert Python programmer and code repair specialist. '
             'Your task is to correct Python SYNTAX errors only.\n'
             'Make ONLY the minimal changes strictly required for the code to '
             'parse and compile successfully.\n'
             'Do NOT fix logic errors, runtime errors, or improve code quality '
             'unless required to resolve a syntax error.\n'
             'Do NOT refactor, reformat, or modify any code that is already '
             'syntactically valid.\n'
             'Preserve all error-free lines exactly as they appear in the '
             'original code.\n'
             'The final output must be a syntactically valid Python program.\n'
             'Output ONLY the corrected Python code.\n'
             'Do NOT include explanations, reasoning, comments, markdown, or '
             'formatting wrappers of any kind.\n'
             'Do NOT add any text before or after the code.',
  'role': 

In [28]:
print(train_dataset.features)
print(val_dataset.features)

{'messages': List({'content': Value('string'), 'role': Value('string')})}
{'messages': List({'content': Value('string'), 'role': Value('string')})}


In [29]:
import os
import time

output_dir = f"finetuned_models/{model_name}/code_llama_fine_tune_reason_{int(time.time())}"
os.makedirs(f"{output_dir}", exist_ok=True)

In [31]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,   # ← REQUIRED
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    formatting_func = formatting_func,
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 15,
        eval_strategy = "steps",
        eval_steps = 100,
        save_strategy = "steps",
        save_steps = 100,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = output_dir,
        logging_strategy = "steps",
        logging_steps = 50,
        logging_first_step = True,
        load_best_model_at_end = True,
        metric_for_best_model = "eval_loss",
        greater_is_better = False,
        save_total_limit = 2,
        report_to = "none",
    ),
)


Unsloth: Tokenizing ["text"] (num_proc=24): 100%|██████████| 116/116 [00:03<00:00, 31.75 examples/s]


In [32]:
import gc
torch.cuda.empty_cache()
gc.collect()

283

In [33]:
trainer_stats = trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,036 | Num Epochs = 15 | Total steps = 975
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 134,217,728 of 32,898,094,080 (0.41% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss
100,0.448400,0.302160
200,0.285500,0.276411
300,0.231200,0.275586
400,0.209600,0.284341
500,0.199800,0.285060
600,0.187200,0.316142
700,0.161400,0.326957
800,0.131000,0.374417
900,0.092100,0.410556


Unsloth: Not an error, but Qwen2ForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


In [34]:
pprint(trainer_stats)

TrainOutput(global_step=975, training_loss=0.23818279657608424, metrics={'train_runtime': 82549.5439, 'train_samples_per_second': 0.188, 'train_steps_per_second': 0.012, 'total_flos': 3.063921451508736e+18, 'train_loss': 0.23818279657608424, 'epoch': 15.0})


In [35]:
trainer.state.best_model_checkpoint

'finetuned_models/unsloth/Qwen2.5-Coder-32B-Instruct/code_llama_fine_tune_reason_1768154208/checkpoint-300'

In [36]:
model = model.merge_and_unload()

/home/mxs220189/miniconda3/envs/pyllmpatch/lib/python3.13/site-packages/peft/tuners/lora/bnb.py:397: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [37]:
model.save_pretrained("merged_model_qwen2.5_coder_32b_instruct_15_epochs")
tokenizer.save_pretrained("merged_model_qwen2.5_coder_32b_instruct_15_epochs")

('merged_model_qwen2.5_coder_32b_instruct_15_epochs/tokenizer_config.json', 'merged_model_qwen2.5_coder_32b_instruct_15_epochs/special_tokens_map.json', 'merged_model_qwen2.5_coder_32b_instruct_15_epochs/chat_template.jinja', 'merged_model_qwen2.5_coder_32b_instruct_15_epochs/vocab.json', 'merged_model_qwen2.5_coder_32b_instruct_15_epochs/merges.txt', 'merged_model_qwen2.5_coder_32b_instruct_15_epochs/added_tokens.json', 'merged_model_qwen2.5_coder_32b_instruct_15_epochs/tokenizer.json')